In [1]:
import os
import matplotlib.pyplot as plt
import pydicom
import numpy as np
from glob import glob
import pandas as pd
from random import shuffle
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.layers. normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
label = []
train_data = []
i = 0
Y = pd.read_csv('SPECT_4_04_2020.csv', usecols=['Subject', 'Group'])#reading 2 columns of CSV
for f in range(1579):
    path = os.path.join(r'E:\SPECT_Final',str(Y['Subject'][f]),r'Reconstructed_DaTSCAN\*\*\*.dcm')
    if(i>1 and str(Y['Subject'][i]) == str(Y['Subject'][i-2])):
        fp = glob(path)[2]
    elif(i>0 and str(Y['Subject'][i]) == str(Y['Subject'][i-1])):
        try:
            fp = glob(path)[1]
        except(IndexError):
            fp = glob(path)[0] 
    else:
        fp = glob(path)[0]
    dataset = pydicom.dcmread(fp) #reading DCM image
    image = np.array(dataset.pixel_array) #converting into numpy array
    image1 = np.zeros(shape = (image.shape))
    #normalisation and binarization of the image and taking 40th slice only
    for j in range(image.shape[1]):
        for k in range(image.shape[2]):
            image1[40,j,k] = np.divide(image[40,j,k],np.max(image[40]))
            if(image1[40,j,k] > 0.70):
                image1[40,j,k] = 1
            else:
                image1[40,j,k] = 0
    #one-hoy encoding
    if(Y['Group'][f] == 'PD'):
        label = (np.array([1, 0]))
    else:
        label = (np.array([0, 1]))
    train_data.append([np.array(image1[40]), label]) #making train_data

In [3]:
shuffle(train_data)

In [4]:
trainImages = np.array([i[0] for i in train_data]).reshape(-1, 109, 91, 1)
trainLabels = np.array([i[1] for i in train_data])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(trainImages, trainLabels, test_size=0.20)

In [6]:
#Model of Alexnet
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(109,91,1), kernel_size=(11,11), strides=(4,4), padding="valid", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(units = 9216, activation = "relu"))

# 2nd Fully Connected Layer
model.add(Dense(units = 4096, activation = "relu"))

# 3rd Fully Connected Layer
model.add(Dense(4096, activation = "relu"))

# Output Layer
model.add(Dense(2, activation = "softmax")) #As we have two classes

In [7]:
adam = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
model.fit(X_train, y_train, batch_size = 8, epochs = 20, verbose = 1)




Epoch 1/20





1263/1263 [==============================] - 131s 104ms/step - loss: 0.2403 - acc: 0.9082
Epoch 2/20
1263/1263 [==============================] - 130s 103ms/step - loss: 0.1277 - acc: 0.9549
Epoch 3/20
1263/1263 [==============================] - 128s 101ms/step - loss: 0.1053 - acc: 0.9667
Epoch 4/20
1263/1263 [==============================] - 128s 101ms/step - loss: 0.0875 - acc: 0.9755
Epoch 5/20
1263/1263 [==============================] - 127s 100ms/step - loss: 0.0849 - acc: 0.9707
Epoch 6/20
1263/1263 [==============================] - 129s 102ms/step - loss: 0.0754 - acc: 0.9731
Epoch 7/20
1263/1263 [==============================] - 132s 104ms/step - loss: 0.0438 - acc: 0.9873
Epoch 8/20
1263/1263 [==============================] - 139s 110ms/step - loss: 0.0370 - acc: 0.9929
Epoch 9/20
1263/1263 [==============================] - 168s 133ms/step - loss: 0.0409 - acc: 0.9850
Epoch 10/20
1263/1263 [==============================] - 146s 115ms/step - loss: 0.

In [9]:
loss, acc = model.evaluate(X_test, y_test, verbose = 0)

In [10]:
print(acc)

0.990506329113924
